In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import spacy
from tqdm import tqdm
import bea

/home/rootdaenib/anaconda3/envs/semanticSearch/lib/python3.10/site-packages/cupy/_environment.py:438: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda116

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
2022-10-12 18:50:43.506043: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow

In [2]:
driver = webdriver.Chrome(executable_path="./chromedriver")

In [3]:
driver.get("https://documentation.softwareag.com/webmethods/api_gateway/yai10-11/10-11_API_Gateway_webhelp/index.html")


In [12]:
links = [href.find_element(By.TAG_NAME, 'a').get_attribute("href") for href in driver.find_element(By.ID, "toc_content").find_elements(By.TAG_NAME, "ul")]


In [13]:
links = [link for link in links if link is not None]

In [14]:
links = links[3:]

In [7]:
dataJson = {}

In [8]:
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_trf")

In [15]:
links

['https://documentation.softwareag.com/webmethods/api_gateway/yai10-11/10-11_API_Gateway_webhelp/api-gateway-integrated-webhelp/co-api_gateway_overview.html#',
 'https://documentation.softwareag.com/webmethods/api_gateway/yai10-11/10-11_API_Gateway_webhelp/api-gateway-integrated-webhelp/ta-qsg-install-apigw.html#',
 'https://documentation.softwareag.com/webmethods/api_gateway/yai10-11/10-11_API_Gateway_webhelp/api-gateway-integrated-webhelp/ta-qsg_launch_apig.html#',
 'https://documentation.softwareag.com/webmethods/api_gateway/yai10-11/10-11_API_Gateway_webhelp/api-gateway-integrated-webhelp/ta-qsg_create_api.html#',
 'https://documentation.softwareag.com/webmethods/api_gateway/yai10-11/10-11_API_Gateway_webhelp/api-gateway-integrated-webhelp/ta-qsg_configure_destinations.html#',
 'https://documentation.softwareag.com/webmethods/api_gateway/yai10-11/10-11_API_Gateway_webhelp/api-gateway-integrated-webhelp/ta-qsg_request_api_key.html#',
 'https://documentation.softwareag.com/webmethods

In [49]:
dataJson = {}
exceptionLinks = list()
tqdmLinks = tqdm(links)
for link in tqdmLinks:
    try:
        driver.get(link)
        driver.switch_to.frame(0)
        title = driver.find_element(By.ID, "wwconnect_header").text
        if title not in dataJson:
            dataJson[title] = {}
        driver.execute_script("""
            var l = document.getElementById("wwconnect_header");
            l.parentNode.removeChild(l);
            """)
        divs = driver.find_elements(By.TAG_NAME, "div")
        dataJson[title]['data'] = []
        for div in divs[:-1]:
            if div.text != '':
                doc = nlp(div.text)
                for sent in doc.sents:
                    dataJson[title]['data'].append(sent.text)
        dataJson[title]['href'] = link
    except Exception as e:
        print(e)
        print(link)
        exceptionLinks.append(link)
        # break

100%|██████████| 208/208 [07:13<00:00,  2.08s/it]


In [51]:
import json
with open("data.json", "w") as f:
    json.dump(dataJson, f)
with open("exceptionLinks.json", "w") as f:
    json.dump(exceptionLinks, f)

In [50]:
len(dataJson)

208

In [20]:
exceptionLinks

[]

In [21]:
len(links)

208

In [33]:
driver.switch_to.parent_frame()
driver.find_element(By.TAG_NAME, 'title').text

''

In [38]:
print(driver.execute_script("return document.title"))

API Gateway Cluster Configuration
